In [ ]:
!pip install langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain-google-genai google-generativeai

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

# Access the API key from Colab's Secrets Manager
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [81]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
import requests

In [82]:
@tool
def get_current_weather(city:str) -> str:
  """ Get the current weather in a given city"""
  url = f"https://api.weatherstack.com/current?access_key=fd47af7d136a960dd5f1d99aff1c5dad&query={city}"
  response = requests.get(url)
  return response.json()

In [ ]:
search_tool = DuckDuckGoSearchRun()

In [80]:
results = search_tool.invoke("world news")
results

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'Read the latest headlines, breaking news, and videos at APNews.com, the definitive source for independent journalism from every corner of the globe. World news today: Stay informed with the latest international news and live updates on breaking global events from The Hindu. Explore in-depth coverage of stories from across the globe, … Stay updated with the latest global news and developments on this comprehensive platform covering diverse topics from politics, economy to cultural events worldwide. 2 days ago · Stay on top of News latest developments on the ground with Al Jazeera’s fact-based news, exclusive video footage, photos and updated maps. Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com'

In [ ]:
results

'Over the centuries, different dialects of the Nepali language with distinct influences from Sanskrit, Maithili, Hindi, and Bengali are believed to have emerged across different regions of the … You can use our online Nepali input tool to transliterate unlimited Nepali words for FREE. Our online software is supported on both desktop and mobile devices such as Apple iPhone, … Jun 27, 2025 · Nepali language, member of the Pahari subgroup of the Indo-Aryan group of the Indo-Iranian division of the Indo-European languages. Nepali is spoken by more than 17 … Nepali is a member of the Eastern Pahari branch of the Indo-Aryan language family. It is spoken mainly in Nepal and northern India by about 32 million people, 19 million of whom speak it as a … Feb 16, 2025 · Nepali is a language in the Indo-Aryan family, which is primarily spoken in Nepal. Nepali is also spoken in the eastern parts of India, and in Myanmar (Burma) and Bhutan.'

In [ ]:
!pip install ddgs

In [ ]:
from ddgs import DDGS
search_tool = DDGS()
result = search_tool.text("todays date in nepali?")

In [ ]:
(result)[1]

{'title': 'Today Nepali Date and Time - Ankjit',
 'href': 'https://ankjit.com.np/today-nepali-date-and-time/',
 'body': 'Nov 28, 2023 · We have developed a dynamic Nepali calendar where you can see today’sNepali month, day, and current Nepali time. You can bookmark this page to check Nepalidates and times daily.'}

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

# AGENT

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


In [ ]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


🔗 [hwchase17/react DOCS](https://smith.langchain.com/hub/hwchase17/react)

In [ ]:
from langchain_core.tools import Tool
agent = create_react_agent(
    llm = llm,
    tools=[Tool(name="duckduckgo_search", func=search_tool.text, description="A tool to perform web searches.")],
    prompt=prompt
)

In [ ]:
# wrap the agent with the agent executor

agent_executor = AgentExecutor(
    agent=agent,
    tools=[Tool(name="duckduckgo_search", func=search_tool.text, description="A tool to perform web searches.")],
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
# invoke
response = agent_executor.invoke({"input":"what is the temperature of nepal"})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to find the current temperature in Nepal.  This will likely vary by location, so I should search for a general average or specify a major city.  Let's try Kathmandu first.
Action: duckduckgo_search
Action Input: "temperature in Kathmandu, Nepal"[{'title': 'Kathmandu (Nepal) weather - Met Office', 'href': 'https://weather.metoffice.gov.uk/forecast/tuuttd8hh', 'body': '5 days ago · Kathmandu7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV'}, {'title': 'Weather today - Kathmandu, Nepal', 'href': 'https://www.weather-atlas.com/en/nepal/kathmandu', 'body': '1 day ago · Kathmandu, Nepal - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data.'}, {'title': 'Kathmandu, Nepal 14 day weather forecast - timeanddate.com', 'href': 'h

In [ ]:
response

{'input': 'what is the temperature of nepal',
 'output': 'The temperature in Kathmandu, Nepal is currently 28°C according to Accuweather.  Note that this is a single point measurement and the temperature may vary across Nepal.'}

**in simple**

In [ ]:
search_tool = DuckDuckGoSearchRun()

In [83]:
@tool
def get_current_weather(city:str) -> str:
  """ Get the current weather in a given city"""
  url = f"https://api.weatherstack.com/current?access_key=fd47af7d136a960dd5f1d99aff1c5dad&query={city}"
  response = requests.get(url)
  return response.json()

In [84]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [85]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [86]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [87]:
# create the ReAct agent manually with the pulled prompt
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(
    llm = llm,
    tools=[search_tool,get_current_weather],
    prompt=prompt
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [88]:
# wrap it with the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_current_weather],
    verbose=True
)

In [89]:
# invoke
response = agent_executor.invoke({"input":"what is the temperature of nepal, biratnagar city"})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to get the current weather in Biratnagar, Nepal.
Action: get_current_weather
Action Input: city='Biratnagar'{'request': {'type': 'City', 'query': 'Biratnagar, Nepal', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Biratnagar', 'country': 'Nepal', 'region': '', 'lat': '26.483', 'lon': '87.283', 'timezone_id': 'Asia/Kathmandu', 'localtime': '2025-07-14 13:02', 'localtime_epoch': 1752498120, 'utc_offset': '5.75'}, 'current': {'observation_time': '07:17 AM', 'temperature': 32, 'weather_code': 176, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0009_light_rain_showers.png'], 'weather_descriptions': ['Patchy rain nearby'], 'astro': {'sunrise': '05:12 AM', 'sunset': '06:51 PM', 'moonrise': '09:28 PM', 'moonset': '08:18 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 89}, 'air_quality': {'co': '525.4', 'no2': '11.47', 'o3': '87', 'so2': '4.44', 'pm2_5': '26.27', 'pm10': '26.64', 'us

# ReACt Design pattern

🔗 [medium](https://medium.com/google-cloud/building-react-agents-from-scratch-a-hands-on-guide-using-gemini-ffe4621d90ae)